In [482]:
import pandas as pd
import pickle
from scipy.stats import poisson
import math
import random

### Leer Data

In [483]:
with open("data/dict_grupos.pkl","rb") as archivo:
    dict_grupos2022 = pickle.load(archivo)
archivo.close()

df_FIFA_Historical = pd.read_csv("data/FIFAWC.csv")
df_Qatar = pd.read_csv("data/clean_Qatar.csv")

In [484]:
df_Qatar

,Unnamed: 0,Local,Visitante,Marcador,Año
0,0,Catar,Ecuador,Partido 1,2022
1,1,Senegal,Países Bajos,Partido 2,2022
2,2,Catar,Senegal,Partido 18,2022
3,3,Países Bajos,Ecuador,Partido 19,2022
4,4,Ecuador,Senegal,Partido 35,2022
...,...,...,...,...,...
59,59,Ganador partido 51,Ganador partido 52,Partido 59,2022
60,60,Ganador partido 57,Ganador partido 58,Partido 61,2022
61,61,Ganador partido 59,Ganador partido 60,Partido 62,2022
62,62,Perdedor partido 61,Perdedor partido 62,Partido 63,2022


In [485]:
df_Qatar["Local"][0]

'Catar'

In [486]:
print(dict_grupos2022["Grupo G"])

   Pos Selección  Pts  PJ  PG  PE  PP  GF  GC  Dif
0    1    Brasil    0   0   0   0   0   0   0    0
1    2    Serbia    0   0   0   0   0   0   0    0
2    3     Suiza    0   0   0   0   0   0   0    0
3    4   Camerún    0   0   0   0   0   0   0    0


In [487]:
dict_grupos2022["Grupo A"].dtypes

Pos           int64
Selección    object
Pts           int32
PJ            int32
PG            int32
PE            int32
PP            int32
GF            int32
GC            int32
Dif           int32
dtype: object

In [488]:
df_FIFA_Historical

,Local,Visitante,Año,GolesLocal,GolesVisita,GolesTotales
0,Francia,México,1930,4,1,5
1,Argentina,Francia,1930,1,0,1
2,Chile,México,1930,3,0,3
3,Chile,Francia,1930,1,0,1
4,Argentina,México,1930,6,3,9
...,...,...,...,...,...,...
895,Rusia,Croacia,2018,2,2,4
896,Francia,Bélgica,2018,1,0,1
897,Croacia,Inglaterra,2018,2,1,3
898,Bélgica,Inglaterra,2018,2,0,2


In [489]:
## Extraer data de local y visitante por separado
df_Local = df_FIFA_Historical[["Local","GolesLocal","GolesVisita"]].copy()
df_visita = df_FIFA_Historical[["Visitante","GolesLocal","GolesVisita"]].copy()
df_Local

,Local,GolesLocal,GolesVisita
0,Francia,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
895,Rusia,2,2
896,Francia,1,0
897,Croacia,2,1
898,Bélgica,2,0


In [490]:
##3 Renombrar columnas
df_Local.rename(columns={"Local":"Equipo", "GolesLocal":"GolesAnotados","GolesVisita":"GolesRecibidos"},inplace=True)
df_visita.rename(columns={"Visitante":"Equipo", "GolesLocal":"GolesRecibidos","GolesVisita":"GolesAnotados"},inplace=True)
df_visita

,Equipo,GolesRecibidos,GolesAnotados
0,México,4,1
1,Francia,1,0
2,México,3,0
3,Francia,1,0
4,México,6,3
...,...,...,...
895,Croacia,2,2
896,Bélgica,1,0
897,Inglaterra,2,1
898,Inglaterra,2,0


In [491]:
## Concatenamos en un dataframe
equipos = pd.concat([df_Local,df_visita],ignore_index=True)

### Agrupamos por equipo y obtenemos el promedio de cada columna
fuerza_equipos = equipos.groupby(["Equipo"]).mean()
fuerza_equipos

,GolesAnotados,GolesRecibidos
Equipo,,
Alemania,2.021277,1.021277
Alemania Democrática,0.833333,0.833333
Alemania Federal,2.112903,1.241935
Angola,0.333333,0.666667
Arabia Saudita,0.687500,2.437500
...,...,...
Ucrania,1.000000,1.400000
Unión Soviética,1.709677,1.096774
Uruguay,1.553571,1.321429


In [492]:
fuerza_equipos.loc["Irán",]
fuerza_equipos.loc["Estados Unidos",]

GolesAnotados     1.121212
GolesRecibidos    1.878788
Name: Estados Unidos, dtype: float64

In [493]:
fuerza_equipos.index

Index(['Alemania', 'Alemania Democrática', 'Alemania Federal', 'Angola',
       'Arabia Saudita', 'Argelia', 'Argentina', 'Australia', 'Austria',
       'Bolivia', 'Bosnia y Herzegovina', 'Brasil', 'Bulgaria', 'Bélgica',
       'Camerún', 'Canadá', 'Checoslovaquia', 'Chile', 'China', 'Colombia',
       'Corea del Norte', 'Corea del Sur', 'Costa Rica', 'Costa de Marfil',
       'Croacia', 'Cuba', 'Dinamarca', 'Ecuador', 'Egipto', 'El Salvador',
       'Emiratos Árabes Unidos', 'Escocia', 'Eslovaquia', 'Eslovenia',
       'España', 'Estados Unidos', 'Francia', 'Gales', 'Ghana', 'Grecia',
       'Haití', 'Honduras', 'Hungría', 'Indias Orientales Neerlandesas',
       'Inglaterra', 'Irak', 'Irlanda', 'Irlanda del Norte', 'Irán',
       'Islandia', 'Israel', 'Italia', 'Jamaica', 'Japón', 'Kuwait',
       'Marruecos', 'México', 'Nigeria', 'Noruega', 'Nueva Zelanda', 'Panamá',
       'Paraguay', 'Países Bajos', 'Perú', 'Polonia', 'Portugal',
       'RF Yugoslavia', 'República Checa', 'Rumania

In [494]:
def Poisson(x,l):
    return (l**x * math.e**(-l))/math.factorial(x)

def WinnerFirstWC():
    x = random.randint(0,100)
    if x <= 80:
        return (0,3)
    elif x > 80 and x < 95:
        return(1,1)
    else:
        return (3,0)

In [495]:
def Predecir_Puntos(local, visita):
    if local in fuerza_equipos.index and visita in fuerza_equipos.index:
        lambda_local = fuerza_equipos.loc[local,"GolesAnotados"]*fuerza_equipos.loc[visita,"GolesRecibidos"]
        lambda_visita = fuerza_equipos.loc[visita,"GolesAnotados"]*fuerza_equipos.loc[local,"GolesRecibidos"]

        local = 0
        visita = 0
        empate = 0
        for x in range(9):
            for y in range(9):
                p_local = Poisson(x=x,l=lambda_local)
                p_visita = Poisson(x=y, l = lambda_visita)
                prob = p_local*p_visita

                if x == y:
                    empate += prob
                elif x > y:
                    local += prob
                else:
                    visita += prob

        puntos_local = 3*local + empate
        puntos_visita = 3*visita + empate
        #return (round(puntos_local,0), round(puntos_visita,0))
        return (puntos_local,puntos_visita)

    else:
        if local not in fuerza_equipos.index:
            return(WinnerFirstWC())
        else:
            tupla = WinnerFirstWC()
            return tupla[::-1]


In [496]:
grupos = df_Qatar[:48].copy()
octavos = df_Qatar[48:56].copy()
cuartos = df_Qatar[56:60].copy()
semis = df_Qatar[60:62].copy()
tercer = df_Qatar[62:63].copy()
final = df_Qatar[63:64].copy()

In [497]:
#Obtener partidos por grupo
for grupo in dict_grupos2022.keys():
    equipos_en_grupo = dict_grupos2022[grupo]["Selección"].values
    partidos_grupo = grupos[grupos["Local"].isin(equipos_en_grupo)]

    tabla = dict_grupos2022[grupo]

    #Actualizar la tabla
    for partido,row in partidos_grupo.iterrows():
        local = row["Local"]
        visitante = row["Visitante"]
        puntosLocal, puntosVisitante = Predecir_Puntos(local=local,visita=visitante)
        print(local,puntosLocal,visitante,puntosVisitante)
        tabla.loc[tabla["Selección"] == local, "Pts"] += puntosLocal
        tabla.loc[tabla["Selección"] == visitante, "Pts"] += puntosLocal
        tabla.sort_values("Pts",axis=0,ascending = False,inplace=True)
    
    tabla["Pos"] = [i+1 for i in range(4)]

    print(tabla)


Catar 1 Ecuador 1
Senegal 0.8966631961462876 Países Bajos 1.8963328262857722
Catar 0 Senegal 3
Países Bajos 1.9964498627056055 Ecuador 0.782094698139651
Ecuador 1.194698333312759 Senegal 1.5529159369074768
Países Bajos 3 Catar 0
   Pos     Selección       Pts  PJ  PG  PE  PP  GF  GC  Dif
3    1  Países Bajos  5.893113   0   0   0   0   0   0    0
1    2       Ecuador  4.191148   0   0   0   0   0   0    0
0    3         Catar  4.000000   0   0   0   0   0   0    0
2    4       Senegal  2.091362   0   0   0   0   0   0    0
Inglaterra 2.3872443254352747 Irán 0.4333053734215785
Estados Unidos 0.9356581220875901 Gales 1.8032887219105167
Gales 1.9925454285865458 Irán 0.7122928612086139
Inglaterra 2.2325667086526493 Estados Unidos 0.5922351666568737
Gales 1.0828226633395734 Inglaterra 1.5938821978589228
Irán 0.9466817474589969 Estados Unidos 1.821037199710538
   Pos       Selección       Pts  PJ  PG  PE  PP  GF  GC  Dif
0    1      Inglaterra  5.702634   0   0   0   0   0   0    0
1    2   

In [498]:
print(dict_grupos2022["Grupo A"].loc[dict_grupos2022["Grupo A"]["Pos"] == 1, "Selección"])

3    Países Bajos
Name: Selección, dtype: object


In [499]:
octavos

,Unnamed: 0,Local,Visitante,Marcador,Año
48,48,1.° Grupo A,2.° Grupo B,Partido 49,2022
49,49,1.° Grupo C,2.° Grupo D,Partido 50,2022
50,50,1.° Grupo D,2.° Grupo C,Partido 52,2022
51,51,1.° Grupo B,2.° Grupo A,Partido 51,2022
52,52,1.° Grupo E,2.° Grupo F,Partido 53,2022
53,53,1.° Grupo G,2.° Grupo H,Partido 54,2022
54,54,1.° Grupo F,2.° Grupo E,Partido 55,2022
55,55,1.° Grupo H,2.° Grupo G,Partido 56,2022


In [500]:
### Obtener partidos de octavos

locales = octavos["Local"].str.split(".° ")
for local,equipo in zip(octavos["Local"],locales):
    equipo_local = str(dict_grupos2022[equipo[1]].loc[dict_grupos2022[equipo[1]]["Pos"] == int(equipo[0]), "Selección"].values[0])
    octavos.replace(local,equipo_local,inplace=True)

locales = octavos["Visitante"].str.split(".° ")
for local,equipo in zip(octavos["Visitante"],locales):
    equipo_local = str(dict_grupos2022[equipo[1]].loc[dict_grupos2022[equipo[1]]["Pos"] == int(equipo[0]), "Selección"].values[0])
    octavos.replace(local,equipo_local,inplace=True)

octavos

,Unnamed: 0,Local,Visitante,Marcador,Año
48,48,Países Bajos,Irán,Partido 49,2022
49,49,Argentina,Australia,Partido 50,2022
50,50,Francia,Arabia Saudita,Partido 52,2022
51,51,Inglaterra,Ecuador,Partido 51,2022
52,52,Japón,Canadá,Partido 53,2022
53,53,Suiza,Portugal,Partido 54,2022
54,54,Bélgica,Costa Rica,Partido 55,2022
55,55,Uruguay,Brasil,Partido 56,2022
